In [2]:
#openrocket_interface imports
from math import pi, log, sqrt, exp, cos, sin, radians
import os
from sys import platform as _platform
import xml.etree.ElementTree as ET # xml library
from zipfile import ZipFile

#trajectory imports
from types import SimpleNamespace
import numpy as np
import math
import csv
import copy
from datetime import datetime
import nrlmsise00, pyhwm2014

#optimizer imports
from scipy.optimize import minimize, differential_evolution
import pylab
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel

np.set_printoptions(precision=3) # this line may be deprecated

The purpose of this file is to gather all input variables, design parameters, and actual constants in one place. Many of these remain constant from revision to revision, and usually a change to one of these entails another round of optimization and a new canonical description. In tandem with a canonical description, this file should unambiguously define LV4 to a sufficient level of precision for actual engineering work to be done.

The other programs in this folder then have access to all the same information without us having to pass copious amounts of parameters around. Additionally, all of our required imports can be centralized here. There are also some functions defined here which are of use to multiple programs.


In [ ]:
# Physics, chemistry, and materials
G_N      = 9.80665  # kg.m/s^2     Standard gravity
R_UNIV   = 8314.46261815324 # universal gas constant, J/ (K * kmol)
M_PER_IN = 0.0254 # meters per inch


def Material(name, rho, Sy=None):
    return { 'name': name,
             'rho': rho, # kg/m^3       Density
             'Sy': Sy} # Pa           Yield strength

# Materials
# https://www.aircraftspruce.com/catalog/cmpages/anh4120honeycomb01-01574.php?clickkey=5444217
# note: i've calculated ~395 kg/m^3 for uniform airframe density based on our density/thickness estimates.
#       however, measurement of LV3.1 module says module is around 130 kg/m^3. Not sure why these don't agree.
NOMEX      = Material('Nomex', 48.06)
CRYOGEL    = Material('Cryogel', 160)
FIBERGLASS = Material('Fiberglass', 1850)
ALUM       = Material('Aluminum', 2800.0, 0.270e9)
CFIBER     = Material('Carbon Fiber', 1550.0, 0.450e9)
LOX        = Material('LOX', 1141.0) # kg/m^3  Density of LOX
IPA        = Material('IPA/H20', 849.28) # kg/m^3  Density of 64.8% IPA / 35.2% H20

# Nitrogen characteristics
# https://github.com/psas/reaction-control/blob/master/pubs/AIAA%20RCS%20Manuscript_FINAL2.pdf
N2_TEMP = 293 # K, holding this constant is sketchy but easy
N2_MM   = 28.01 # nitrogen molecular mass [g/mol]
N2_KE   = 1.4 # nitrogen specific heat ratio

MANUAL_PROP_CALCS = True
if MANUAL_PROP_CALCS:
    # combustion gas properties ke, Re, T_ch, determined from CEArun
    # with chamber pressure=350 psi, fuel temp=419.15 K, 
    #      lox temp=90 K, OF=1.3 for fuel = 64.8% IPA (2propanol) / 35.2% H20
    OF       = 1.3        # O/F ratio, this is somewhat arbitrary but CEA says its good. 
    ENG_P_CH = 2413166 # chamber pressure, PSI
    ENG_T_CH = 3097.82 # chamber temperature, K
    ENG_KE   = 1.1251 # specific heat ratio, propellant (aka gammas)
    ENG_MM   = 23.196 # molar mass
else:
    pass


In [ ]:
# Launch constants
LAUNCH_SITE_ALT = 1401 # m, altitude of launch site above sea level
LAUNCH_TOWER    = 18.288 # launch rail height in m
LAUNCH_SITE_LOC = [32.9895472, -106.9694681] # dec deg N, E

In [ ]:
# System Definition

# Rocket parameters
AIRFRM_IN_RAD   = 0.14414 #0.1524 # rocket inner radius, m
TANK_IN_RAD     = 0.1397 # propellant tank inner radius, m
TANK_EQV_WT_T   = 0.00236 # tank equivalent weight thickness, m
THROTTLE_WINDOW = (100., 500.) # lower and upper bounds of drag force (N) for throttling
MIN_THROTTLE    = 1. # the internet says between 60 - 70% is doable without ruining our lives.
NOSETIP         = 0.6 # 0.6 kg estimated for aluminum nose tip. Add more as needed for stability.

# Fin Geometry
FIN_ROOT        = 0.7 # Root length, m
FIN_TIP         = 0.45 # tip length, m
FIN_SWEEP_ANGLE = np.radians(70) # sweep angle, degrees
FIN_SEMISPAN    = 0.42 # fin span/height, m
FIN_THICKNESS   = 0.003175 # m, fin thickness from LV3.1
FIN_ROOT_HEIGHT = 0.082525 # height of bottom of fin root above base of rocket

# RCS parameters
# low priority, find constraints for these and optimize
RCS_CONTROL = False # whether sims have RCS enabled
RCS_MDOT    = 0.03784/3.15 # kg/s RCS nozzle mass flow rate
RCS_P_E     = 101300 # Pa, RCS nozzle exit pressure
RCS_P_CH    = 689476 # Pa, RCS chamber pressure
N2_TANK_P   = 3.103e7 # Pa, N2 tank pressure

# upper subsystem module dimensions, if these change you will have to make changes in structure.ipynb
AIRFRAME_THICKNESS = 0.008255 # m, (0.325 in)
COUPLING_RING_HT   = 0.03505 # m, space between module
CON_NOSE_L         = 1.5
CYL_NOSE_L         = 0.3048
NOSE_L             = CON_NOSE_L + CYL_NOSE_L       # m, with 1 foot cylinder at end
ERS_L              = 4.44 * M_PER_IN  # m 
RCS_L              = 4.44 * M_PER_IN  # m 
AV_L               = 22.62 * M_PER_IN # m
N2_L               = 22.62 * M_PER_IN # m 
PAS_L              = 4.44 * M_PER_IN  # m 
FIN_CAN_L          = 32.62 * M_PER_IN # m, height of fin can module

# Engine system dimensions. 'm_' = mass, 'l_' = length
# when you have time, these masses should transfer out of structures.ipynb and go into openrocket_interface.ipynb
ULLAGE             = 1.1          # percentage of length added to a tank to account for not filling
L_FEED             = 0.4572       # m, this is 18"
L_EMS              = 0.1016       # m, this is 4" 
L_ENGINE           = 0.300        # m

LOX_TANK_P         = 689476 # Pa, lox tank pressure (100 PSI)
IPA_TANK_P         = 689476 # Pa, ipa tank pressure (100 PSI)

In [ ]:
# SIMULATION AND OPTIMIZATION PARAMETERS
DELTA      = 10**(-3)  # a guess at a good margin for design "convergence"
MU_0       = 0.0025    # barrier parameter, this value lets altitudes approach lower bound pretty quickly
RHO_0      = 5         # penalty parameter, i'm still playing with this value.
DT         = 0.05      # change starting time-step for trajectory simulation
ITERATIONS = 1         # number of escalating iteration sequences

# INITIAL DESIGN GUESS
# be sure that you start with a feasible design, otherwise the problem will be ill-conditioned
M_PROP = 167.764   # propellant mass (kg)
MDOT   = 3.35    # Propellant mass flow rate (kg/s)
P_E    = 77236.088 # Exit Pressure (Pa)

X0 = np.array([M_PROP, MDOT, P_E, 0, FIN_ROOT, FIN_TIP, FIN_SWEEP_ANGLE, FIN_SEMISPAN, FIN_THICKNESS]) # initial design vector

# OPTIMIZATION CONSTRAINTS
CONS_IMPLS   = 889600                    # maximum impulse, N s
CONS_AOA     = 10.                       # maximum angle of attack
CONS_MASS    = 270.                      # GLOW constraint, kg, somewhat arbitrary
CONS_LS      = 22.                       # min launch speed from 60' tower constraint, m/s
CONS_TWR     = 2.                        # TWR constraint
CONS_S_CRIT  = 0.35                      # Critical pressure ratio constraint
CONS_ACCEL   = 15.                       # Max acceleration constraint, g's
CONS_LD      = 22.                       # L/D ratio constraint, slightly arbitrary
CONS_ALT     = 100000.                   # Min altitude constraint, m
CONS_THRUST  = 6.                        # max ground-level thrust, kN
CONS_CEILING = 150000.                   # base-11 maximum apogee requirement, km
CONS_STBLTY  = 2.0                       # minimum in flight stability margin caliber

In [ ]:
RKT_PREFIX = "../rocket_farm/" # the rockets live on a cute little farm upstate where they frolic in fields

## Utility Functions
# unpack rocket template temporarily
def unzip():
    with ZipFile('../LV4_canonical/template.ork') as myzip:
        myzip.extract('rocket.ork')

# package our new rocket and remove temporary template
def zipit(index):
    with ZipFile(RKT_PREFIX+'psas_rocket_'+index+'.ork', 'w') as myzip:
        myzip.write('rocket.ork')
    if 'linux' in _platform:
        os.system('rm rocket.ork')
    elif "darwin" in _platform:
        os.system('rm rocket.ork')
    elif "win" in _platform:
        os.system('del rocket.ork')

# pulls ALL file references from given directory
def all_files(directory):
    for path, dirs, files in os.walk(directory):
        for f in sorted(files):
            yield os.path.join(path, f)

# counts how many rockets are in our directory and then increments by 1
def get_index():
    ork_files = [f for f in all_files(RKT_PREFIX)
                   if f.endswith('.ork')]
    return len(ork_files) + 1


In [ ]:
# Consider that there are two tanks, and we will want to divide total mass flow rate and propellant mass
# oxygen first, fuel second
def proportion(amount, OF):
    stuff_o = amount * OF/(1 + OF)
    stuff_f = amount * 1/(1 + OF)
    return stuff_o, stuff_f
    
# this is hamilton's quaternion product
def product(a, b):
    v = b[0] * a[1:] + a[0] * b[1:] + np.cross(a[1:], b[1:])
    return np.array([a[0] * b[0] - np.dot(a[1:], b[1:]), v[0], v[1], v[2]])

# this is the inverse of a unit quat
def conjugate(q):
    return np.array([q[0], -q[1], -q[2], -q[3]])

# this rotates a vector with a fixed frame
def sandwich(q, v):
    return product(q, product(np.array([0, v[0], v[1], v[2]]), conjugate(q)))[1:]

# this rotates a frame with a fixed vector
def frame_rotation(q, v):
    return sandwich(conjugate(q), v)

# this constrains a quat to S3 or vector to S2
def normalize(q):
    norm = np.linalg.norm(q)
    norm = norm if norm !=0 else 1
    return q / norm

def eulerangle_to_quat(RA, dec, orientation):
    '''Encodes star tracker's attitude representation as a quaternion in
    3-2-1 order (yaw, pitch, roll). This quaternion transforms the inertial frame to the body frame.

    :params: right ascenscion, declination, roll.
    :returns: Quaternion representation of attitude.'''
    RA = RA / 2
    dec = dec / 2
    ortn = orientation / 2
    c_phi = np.cos(ortn)
    s_phi = np.sin(ortn)
    c_theta = np.cos(dec)
    s_theta = np.sin(dec)
    c_psi = np.cos(RA)
    s_psi = np.sin(RA)
    return np.array([c_phi * c_theta * c_psi  +  s_phi * s_theta * s_psi,
                     s_phi * c_theta * c_psi  -  c_phi * s_theta * s_psi,
                     c_phi * s_theta * c_psi  +  s_phi * c_theta * s_psi,
                     c_phi * c_theta * s_psi  -  s_phi * s_theta * c_psi
                     ])